<a href="https://colab.research.google.com/github/sseyonafre/Project/blob/main/AI_14_%EA%B3%A0%EC%84%B8%ED%9D%AC_Section3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving user1.csv to user1.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv('user.csv',index_col = 1) 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15312 entries, 1 to 15318
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ageClass          15312 non-null  int64  
 1   ageDegree         15312 non-null  int64  
 2   ageGbn            15312 non-null  object 
 3   certGbn           15312 non-null  object 
 4   height            15312 non-null  float64
 5   weight            15312 non-null  float64
 6   crunch            15312 non-null  object 
 7   jump              15312 non-null  object 
 8   trunkFlexion      15312 non-null  object 
 9   IllinoisAgility   15312 non-null  object 
 10  BMI               15312 non-null  float64
 11  situp             15312 non-null  object 
 12  standinglongjump  15312 non-null  object 
 13  standsit          15312 non-null  object 
 14  twominwalk        15312 non-null  object 
 15  threeMwalk        15312 non-null  object 
 16  exercise          15312 non-null  object

# **EDA**

In [ ]:
df.drop_duplicates(keep='first', inplace=True)
df.replace('-',np.nan,inplace=True)
for i in ['jump','situp','crunch','trunkFlexion','IllinoisAgility','standinglongjump','twominwalk','threeMwalk']:
    df[i]= df[i].astype(float)
df['certGbn'] = df['certGbn'].astype('category')
df.certGbn.replace({'참가증':'4','1등급':'1','2등급':'2','3등급':'3'},inplace=True)

In [ ]:
df1 = df.query("ageGbn=='노인'")
df2 = df.query("ageGbn=='성인'")
df3 = df.query("ageGbn=='유소년'")
df4 = df.query("ageGbn=='청소년'")

df1.drop(columns=['id','crunch','jump','IllinoisAgility','situp','standinglongjump'],inplace=True)
df2.drop(columns=['id','crunch','jump','IllinoisAgility','standsit','twominwalk','threeMwalk'],inplace=True)
df3.drop(columns=['id','jump','situp','standsit','twominwalk','threeMwalk'],inplace=True)
df4.drop(columns=['id','crunch','situp','standsit','twominwalk','threeMwalk'],inplace=True)

df1.standsit = df1.standsit.replace(np.nan,0)
df2.situp = df2.situp.replace(np.nan,0).round(0).astype(int)
df3.crunch = df3.crunch.replace(np.nan,0).round(0).astype(int)
df4.jump = df4.jump.replace(np.nan,0).round(0).astype(int)

# **모델링**
> ## 나이구분에 따라 검사항목이 다르기 때문에 모델을 나이구분별로 만들어 준다.
### 1. 노인층 모델링
> 기준모델

In [ ]:
#노인층의 모델.

In [ ]:
#train, test 셋으로 분리 , target 설정.

target = 'certGbn'

train, test = train_test_split(df1, train_size=0.80, test_size=0.20, 
                              stratify=df1[target], random_state=2)
features1 = df1.drop(columns=['certGbn','exercise','testYm','ageGbn']).columns

X_train1 = train[features1]
y_train1 = train[target]
X_test1 = test[features1]
y_test1 = test[target]

In [ ]:
#기준모델 
y_train1.value_counts(normalize=True) #불균형. 다중분류.

4    0.539503
3    0.209932
2    0.182844
1    0.067720
Name: certGbn, dtype: float64

In [ ]:
#기준모델을 타겟의 최빈값으로 설정.
baseline1 = y_train1.mode()

In [ ]:
#기준모델의 정확도를 구해보자.
y_pred = [baseline1] * len(y_test1)
print("기준모델 정확도: ", accuracy_score(y_test1, y_pred))

기준모델 정확도:  0.5405405405405406


> 로지스틱회귀모델

In [ ]:
#로지스틱회귀모델을 만들어 교차검증을 수행. #베이지안최적화를 사용해서 최적의 파라미터 찾아보자.

pipe1_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe1_lr, X_train1, y_train1, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.71166156 0.6990203  0.63592587 0.6282492  0.71689419]
교차검증 f1 평균: 0.6783502236628092


> 랜덤포레스트모델

In [ ]:
pipe1_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe1_rfc, X_train1, y_train1, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.66001786 0.67351525 0.6713029  0.67623248 0.6945366 ]
교차검증 f1 평균: 0.675121015442505


> XGBoost모델


In [ ]:

pipe1_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)

pipe1_xg.fit(X_train1,y_train1)

k = 5

scores = cross_val_score(pipe1_xg, X_train1, y_train1, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.81649732 0.66315015 0.69207608 0.71975095 0.78686929]
교차검증 f1 평균: 0.7356687579782959


### 2. 성인층 모델링
> 기준모델

In [ ]:
#train, test 셋으로 분리 , target 설정.

train, test = train_test_split(df2, train_size=0.80, test_size=0.20, 
                              stratify=df2[target], random_state=2)
features2 = df2.drop(columns=['certGbn','exercise','testYm','ageGbn']).columns

X_train2 = train[features2]
y_train2 = train[target]
X_test2 = test[features2]
y_test2 = test[target]

In [ ]:
#기준모델 
y_train2.value_counts(normalize=True) #불균형. 다중분류.

4    0.519111
3    0.208268
2    0.186427
1    0.086193
Name: certGbn, dtype: float64

In [ ]:
#기준모델을 타겟의 최빈값으로 설정.
baseline2 = y_train2.mode()
y_pred = [baseline2] * len(y_test2)
print("기준모델 정확도: ", accuracy_score(y_test2, y_pred))

기준모델 정확도:  0.5195007800312013


> 로지스틱회귀모델

In [ ]:
pipe2_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe2_lr, X_train2, y_train2, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.63215804 0.65909856 0.65152589 0.63362037 0.65374791]
교차검증 f1 평균: 0.6460301523670579


> 랜덤포레스트모델

In [ ]:
pipe2_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe2_rfc, X_train2, y_train2, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.73515494 0.76101778 0.70984374 0.68029178 0.73338485]
교차검증 f1 평균: 0.7239386173061971


> XGBoost모델

In [ ]:
pipe2_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)

pipe2_xg.fit(X_train2,y_train2)

k = 5

scores = cross_val_score(pipe2_xg, X_train2, y_train2, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.74094186 0.75157466 0.69427815 0.69124133 0.74845691]
교차검증 f1 평균: 0.7252985837606745


### 3. 유소년 모델링
> 기준모델

In [ ]:
#train, test 셋으로 분리 , target 설정.
train, test = train_test_split(df3, train_size=0.80, test_size=0.20, 
                              stratify=df3[target], random_state=2)
features3 = df3.drop(columns=['certGbn','exercise','testYm','ageGbn']).columns

X_train3 = train[features3]
y_train3 = train[target]
X_test3 = test[features3]
y_test3 = test[target]

y_train3.value_counts(normalize=True)

baseline3 = y_train3.mode()
y_pred = [baseline3] * len(y_test3)
print("기준모델 정확도: ", accuracy_score(y_test3, y_pred))

기준모델 정확도:  0.7662337662337663


> 로지스틱회귀모델

In [ ]:
pipe3_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe3_lr, X_train3, y_train3, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.72533878 0.72811088 0.75015845 0.71917467 0.7237942 ]
교차검증 f1 평균: 0.7293153956828793


> 랜덤포레스트모델

In [ ]:
pipe3_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe3_rfc, X_train3, y_train3, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.82574584 0.80756332 0.82372329 0.80589826 0.78731095]
교차검증 f1 평균: 0.8100483320883903


> XGBoost모델

In [ ]:
pipe3_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)

pipe3_xg.fit(X_train3,y_train3)

k = 5

scores = cross_val_score(pipe3_xg, X_train3, y_train3, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.82420439 0.82751193 0.83766176 0.80734666 0.83019189]
교차검증 f1 평균: 0.8253833246659589


###4. 청소년 모델링
> 기준모델

In [ ]:

train, test = train_test_split(df4, train_size=0.80, test_size=0.20, 
                              stratify=df4[target], random_state=2)
features4 = df4.drop(columns=['certGbn','exercise','testYm','ageGbn']).columns

X_train4 = train[features4]
y_train4 = train[target]
X_test4 = test[features4]
y_test4 = test[target]

y_train4.value_counts(normalize=True)

baseline4 = y_train4.mode()
y_pred = [baseline4] * len(y_test4)
print("기준모델 정확도: ", accuracy_score(y_test4, y_pred))

기준모델 정확도:  0.6786556603773585


> 로지스틱회귀모델

In [ ]:
pipe4_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe4_lr, X_train4, y_train4, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.64451804 0.64640353 0.64179533 0.64843367 0.61796486]
교차검증 f1 평균: 0.63982308756246


> 랜덤포레스트모델

In [ ]:
pipe4_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)

pipe4_rfc.fit(X_train4,y_train4)

k = 5

scores = cross_val_score(pipe4_rfc, X_train4, y_train4, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.74467375 0.74647647 0.7433279  0.73286115 0.71733893]
교차검증 f1 평균: 0.7369356383949911


> XGBoost모델

In [ ]:
pipe4_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe4_xg, X_train4, y_train4, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.71948866 0.72393508 0.71866612 0.72611267 0.72132139]
교차검증 f1 평균: 0.7219047845818647


### 결과
 >청소년모델을 제외한 나머지는 xgboost로 청소년은 랜덤포레스트로 결정.



# 모델 피클파일로 저장

In [ ]:
# 해당모델들을 피클라이브러리를 이용하여 피클파일로 저장.

In [ ]:
import pickle 
with open('senior.pickle','wb') as fw:
    pickle.dump(pipe1_xg, fw)

In [ ]:
with open('middle.pickle','wb') as fw:
    pickle.dump(pipe2_xg, fw)

In [ ]:
with open('underaged.pickle','wb') as fw:
    pickle.dump(pipe3_xg, fw)

In [ ]:
with open('youth.pickle','wb') as fw:
    pickle.dump(pipe4_rfc, fw)